# Brazilian E-Commerce - Data Cleansing
> *In this notebook, we are preparing the Brazilian E-Commerce Public Dataset by Olist. The goal is to clean the data strategically to serve as the foundation for an AI Agent (Text-to-SQL) and convert it into a relational SQLite database.*

### 1. Importing Libraries
First, we import the standard data manipulation libraries and `sqlite3`, which will be used later to build our local relational database.

In [1]:
import numpy as np 
import pandas as pd 
import os
import sqlite3

### 2. Exploring the Dataset Directory
Let's map out all the available CSV files in the Olist dataset to identify the core tables we need to build our schema.

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### 3. Loading the Core Business Tables
To build an efficient Text-to-SQL agent, we shouldn't feed it unnecessary tables. 

In [3]:
base_path = 'D:\SEM4\F2 AI\AI project\project\\'

# Carregando apenas as 4 tabelas essenciais
#"D:\SEM4\F2 AI\AI project\project\olist_customers_dataset.csv"
df_customers = pd.read_csv(base_path + 'olist_customers_dataset.csv')
df_sellers = pd.read_csv(base_path + 'olist_sellers_dataset.csv')
df_reviews = pd.read_csv(base_path + 'olist_order_reviews_dataset.csv')
df_items = pd.read_csv(base_path + 'olist_order_items_dataset.csv')
df_products = pd.read_csv(base_path + 'olist_products_dataset.csv')
df_orders = pd.read_csv(base_path + 'olist_orders_dataset.csv')
df_orders_payments = pd.read_csv(base_path + 'olist_order_payments_dataset.csv')

print("Core tables loaded successfully!")

<>:1: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
<>:1: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
C:\Users\Lohet.S.Ratnakar\AppData\Local\Temp\ipykernel_16960\2230460006.py:1: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
  base_path = 'D:\SEM4\F2 AI\AI project\project\\'


Core tables loaded successfully!


### 4. Step-by-Step Data Cleansing

In [4]:
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
df_customers.info()

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   customer_id               99441 non-null  str  
 1   customer_unique_id        99441 non-null  str  
 2   customer_zip_code_prefix  99441 non-null  int64
 3   customer_city             99441 non-null  str  
 4   customer_state            99441 non-null  str  
dtypes: int64(1), str(4)
memory usage: 3.8 MB


In [6]:
df_customers.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

**Conclusion for `df_customers`:** The table is perfectly clean. There are no missing values, and the data types are appropriate for SQL operations (strings for IDs/locations and integers for zip codes). No transformations are required.

In [7]:
df_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [8]:
df_sellers.info()

<class 'pandas.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   seller_id               3095 non-null   str  
 1   seller_zip_code_prefix  3095 non-null   int64
 2   seller_city             3095 non-null   str  
 3   seller_state            3095 non-null   str  
dtypes: int64(1), str(3)
memory usage: 96.8 KB


In [9]:
df_sellers.isnull().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

**Conclusion for `df_sellers`:** The table is clean and ready. There are no missing values in the primary key (`seller_id`) or geographical columns. This guarantees safe `JOIN` operations with the `order_items` table later on.

#### Analyzing the Reviews Table (`df_reviews`)
Customer feedback is a goldmine for business insights. However, in e-commerce, many users leave a star rating without writing a text review. This leads to missing values in the text columns. We need to handle these nulls so the Text-to-SQL agent can accurately query and count reviews based on text presence.

In [10]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [11]:
df_reviews.info()

<class 'pandas.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   review_id                99224 non-null  str  
 1   order_id                 99224 non-null  str  
 2   review_score             99224 non-null  int64
 3   review_comment_title     11568 non-null  str  
 4   review_comment_message   40977 non-null  str  
 5   review_creation_date     99224 non-null  str  
 6   review_answer_timestamp  99224 non-null  str  
dtypes: int64(1), str(6)
memory usage: 5.3 MB


In [12]:
df_reviews.isnull().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

**Action Plan:** We cannot drop the rows with missing text reviews because the `review_score` (1 to 5 stars) is still highly valuable. Instead, we will fill the missing text values with a default string like 'Sem Título' (No Title) and 'Sem Comentário' (No Comment).

In [13]:
# 4. Treating null values strategically for the AI project
df_reviews['review_comment_title'] = df_reviews['review_comment_title'].fillna('Sem Título')
df_reviews['review_comment_message'] = df_reviews['review_comment_message'].fillna('Sem Comentário')

# Verifying the cleanup
print("--- AFTER CLEANUP ---")
print(df_reviews.isnull().sum())

--- AFTER CLEANUP ---
review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64


**Conclusion for `df_reviews`:** Missing texts were successfully replaced with standardized strings. Now, if a manager asks the AI, "How many reviews have no comments?", the agent can easily translate that to `WHERE review_comment_message = 'Sem Comentário'`, preventing SQL errors caused by `NULL` handling.

In [14]:
df_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [15]:
df_items.info()

<class 'pandas.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  str    
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  str    
 3   seller_id            112650 non-null  str    
 4   shipping_limit_date  112650 non-null  str    
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), str(4)
memory usage: 6.0 MB


In [16]:
df_items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

**Conclusion for `df_items`:** There are zero missing values across all its 112,650 rows. Both the foreign keys (`order_id`, `product_id`, `seller_id`) and the financial metrics are fully populated and correctly typed, meaning no further cleaning is required for this table.

#### Analyzing the Products Table (`df_products`)
The products table is central to understanding what is being sold. However, it presents two challenges for the Text-to-SQL agent:
1. Missing category names, which would break `GROUP BY` aggregations.
2. Irrelevant physical dimensions (weight, height, width), which only add noise to the LLM's context window and consume unnecessary tokens.
Let's inspect the data first.

In [17]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [18]:
df_products.info

df_products.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

**Action Plan:** First, we will fill the missing `product_category_name` values with 'Outros' (Others) so the AI can safely group unclassified items. Then, we will drop all physical measurement columns to streamline the database schema, making it highly optimized for the AI Agent.

In [19]:
# 4. Treating null categories
df_products['product_category_name'] = df_products['product_category_name'].fillna('Outros')

# 5. Dropping irrelevant physical/logistical columns to save LLM tokens
measurement_cols = [
    'product_name_lenght', 
    'product_description_lenght', 
    'product_photos_qty', 
    'product_weight_g', 
    'product_length_cm', 
    'product_height_cm', 
    'product_width_cm'
]

df_products = df_products.drop(columns=measurement_cols)

# Verifying the cleanup
print("--- AFTER CLEANUP ---")
print(df_products.isnull().sum())
print("\nRemaining columns for the AI:", df_products.columns.tolist())

--- AFTER CLEANUP ---
product_id               0
product_category_name    0
dtype: int64

Remaining columns for the AI: ['product_id', 'product_category_name']


**Conclusion for `df_products`:** The table is now lean and optimized. Missing categories were categorized as 'Outros', and all unnecessary physical dimensions were removed. The agent will now only see the essential product ID and category, significantly improving its SQL generation accuracy.

#### Analyzing the Orders Table (`df_orders`)
The `df_orders` table tracks the lifecycle of every purchase. It contains crucial timestamps (purchase, approval, delivery). 

**Important Business Logic:** Missing values in delivery dates are expected for canceled or ongoing orders. We must NOT fill or drop these date nulls, as they carry business meaning. Our only concern is ensuring structural integrity: no missing primary or foreign keys (`order_id`, `customer_id`).

In [20]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [21]:
df_orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   order_id                       99441 non-null  str  
 1   customer_id                    99441 non-null  str  
 2   order_status                   99441 non-null  str  
 3   order_purchase_timestamp       99441 non-null  str  
 4   order_approved_at              99281 non-null  str  
 5   order_delivered_carrier_date   97658 non-null  str  
 6   order_delivered_customer_date  96476 non-null  str  
 7   order_estimated_delivery_date  99441 non-null  str  
dtypes: str(8)
memory usage: 6.1 MB


In [22]:
df_orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

**Action Plan:** We will intentionally leave the null values in the timestamp columns untouched, as they represent real-world scenarios (e.g., a canceled order has no delivery date). Our Text-to-SQL agent will handle these via standard SQL conditions (`IS NULL` / `IS NOT NULL`). We will only enforce the removal of any row missing its core relational IDs, just as a safety net.

In [23]:
# 4. Enforcing relational integrity (Safety net for SQL JOINs)
df_orders = df_orders.dropna(subset=['order_id', 'customer_id'])

# Verifying the cleanup
print("--- AFTER CLEANUP ---")
print(f"Total rows remaining: {len(df_orders)}")
print("Relational IDs are intact. Date nulls are preserved for business logic.")

--- AFTER CLEANUP ---
Total rows remaining: 99441
Relational IDs are intact. Date nulls are preserved for business logic.


**Conclusion for `df_orders`:** The table's relational integrity is guaranteed. By preserving the null timestamps, we enable the AI Agent to accurately answer operational questions like "How many orders were canceled before shipping?" by querying for null delivery dates alongside the `order_status` column.

In [24]:
df_orders_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [25]:
df_orders_payments.info()

<class 'pandas.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  str    
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  str    
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), str(2)
memory usage: 4.0 MB


In [26]:
df_orders_payments.isnull().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

**Conclusion for `df_orders_payments`:** The table is completely clean with zero missing values. The data types are perfectly aligned for numerical aggregations (SUM, AVG) by the Text-to-SQL agent. No further transformations are necessary.

### 5. Assembling the AI Brain (Exporting to SQLite)
With all 7 tables individually cleaned and optimized for a Large Language Model, the final step is to consolidate them into a relational SQLite database. This `.db` file will act as the single source of truth for our LangChain application.

In [28]:
# Defining the output path in the Kaggle working directory
db_path = r'D:\SEM4\F2 AI\AI project\project\ecommerce.db'

# Connecting to SQLite (this creates the file)
conn = sqlite3.connect(db_path)

# Converting our 7 cleaned DataFrames into SQL tables
df_customers.to_sql('customers', conn, if_exists='replace', index=False)
df_sellers.to_sql('sellers', conn, if_exists='replace', index=False)
df_reviews.to_sql('reviews', conn, if_exists='replace', index=False)
df_items.to_sql('order_items', conn, if_exists='replace', index=False)
df_products.to_sql('products', conn, if_exists='replace', index=False)
df_orders.to_sql('orders', conn, if_exists='replace', index=False)
df_orders_payments.to_sql('payments', conn, if_exists='replace', index=False)

# Closing the connection to safely save the file
conn.close()

print(f"Success! The database was created at: {db_path}")
print("Mission accomplished! You can now download the 'ecommerce.db' file from the output panel.")


Success! The database was created at: D:\SEM4\F2 AI\AI project\project\ecommerce.db
Mission accomplished! You can now download the 'ecommerce.db' file from the output panel.
